#### Imports

In [ ]:
import numpy as np
import math
import scipy as sp
import scipy.stats as st
import matplotlib.pyplot as plt

#### Designing Disease Processes

##### Random Walk Model Function for Disease Simulation

In [ ]:
# define disease processes as a random walk through symptomatic space
def diseaseSimulation(disease_process, num_samples, initial_uncertainty, uncertainty_tau):
    # each disease process is defined by a [start_value, target_value, average_probability_to_increase_severity] pair which parametrizes their random walk
    disease_processes = {"mild":np.array([0.45, 0, 0.45]), "moderate":np.array([0.5, 0.5, 0.5]), "severe":np.array([0.55, 1, 0.55])}
    selected_process = disease_processes[disease_process]
    start_value = selected_process[0]
    target_value = selected_process[1]
    average_probability_to_increase = selected_process[2]
    
    # preallocate storage of stuff and set initial values
    symptoms_sequence = np.zeros(num_samples)
    symptoms_sequence[0] = start_value
    symptoms_uncertainty = np.zeros(num_samples)
    symptoms_uncertainty[0] = initial_uncertainty
    max_increment = 0.05
    min_increment = 0.01
    
    for time_step in np.arange(num_samples-1):
        
        flag = True
        while flag:
            probability_to_increase = np.random.normal(average_probability_to_increase, symptoms_uncertainty[time_step])
            if probability_to_increase > 0:
                flag = False
        
        temp_random = np.random.random_sample()
        if temp_random <= probability_to_increase:
            sign_of_increment = 1
        else:
            sign_of_increment = -1
        
        abs_increment = max_increment*np.abs(target_value - symptoms_sequence[time_step])
        increment = sign_of_increment*np.maximum(abs_increment, min_increment)
        symptoms_sequence[time_step+1] = symptoms_sequence[time_step] + increment
        if symptoms_sequence[time_step+1] > 1:
            symptoms_sequence[time_step+1] = 1
        elif symptoms_sequence[time_step+1] < 0:
            symptoms_sequence[time_step+1] = 0
        symptoms_uncertainty[time_step + 1] = symptoms_uncertainty[time_step]*math.exp(-1/uncertainty_tau)
    
    return symptoms_sequence, symptoms_uncertainty

##### Symptoms Figure Script

In [ ]:
# number of samples
num_samples = 500

# uncertainty decays exponentially - defined by tau
uncertainty_tau = 250

a, uncertainty = diseaseSimulation("mild", num_samples, 0.1, uncertainty_tau)

b, uncertainty = diseaseSimulation("moderate", num_samples, 0.05, uncertainty_tau)
                                              
c, uncertainty = diseaseSimulation("severe", num_samples, 0.01, uncertainty_tau)

In [ ]:
# symptoms figure plotting
symptoms_fig = plt.figure(figsize = (10,10))
symptoms_fig.patch.set_facecolor("white")
plt.rcParams.update({'font.size': 10})
ax1 = plt.subplot(3,1,1)
plt.plot(a)
plt.ylim((-0.02,1.02))
plt.title("Mild Disease Process with Initial Uncertainty = 0.1", fontsize = 14)
plt.tick_params(axis = 'x', bottom = True, top = False, labelbottom = False)
plt.ylabel("Symptomatic Severity")
ax2 = plt.subplot(3,1,2)
plt.plot(b)
plt.ylim((-0.02,1.02))
plt.title("Moderate Disease Process with Initial Uncertainty = 0.05", fontsize = 14)
plt.tick_params(axis = 'x', bottom = True, top = False, labelbottom = False)
plt.ylabel("Symptomatic Severity")
ax3 = plt.subplot(3,1,3)
plt.plot(c)
plt.ylim((-0.02,1.02))
plt.title("Severe Disease Process with Initial Uncertainty = 0.01", fontsize = 14)
plt.xlabel("Time Step")
plt.ylabel("Symptomatic Severity")
symptoms_fig.suptitle("Examples of Stochastic Symptomatic Progression", y = 0.94, fontsize = 20)
plt.show()
symptoms_fig.savefig("symptom_examples.png", bbox_inches='tight')

#### Deriving an Objective Function

##### Net Cost Function

In [ ]:
# define net cost function using Z-equation in thesis
def netCostCalculation(symptomatic_severity, insurance_charge):
    health_cost = 1 / (1 + np.exp(-10*(symptomatic_severity - 0.5)))
    insurance_cost = np.sqrt(insurance_charge/2)
    net_cost = health_cost + insurance_cost
    return health_cost, insurance_cost, net_cost

##### Net Cost Figure Script

In [ ]:
# setting up net cost matrix to visualize
points = np.linspace(0,1,100)
ins_costs = np.zeros((100,))
health_costs = np.zeros((100,))
net_costs = np.zeros((100,100))
for ins_spot in range(100):
    for health_spot in range(100):      
        c = points[ins_spot]
        s = points[health_spot]
        ins_costs[ins_spot], health_costs[health_spot], net_costs[ins_spot, health_spot] = netCostCalculation(s, c)

In [ ]:
# net cost figure plotting
net_cost_fig = plt.figure(figsize = (10,10))
net_cost_fig.patch.set_facecolor("white")
plt.rcParams.update({'font.size': 16})
plt.contourf(points, points, net_costs, 100)
plt.axis('scaled')
plt.colorbar(label = "Net Cost", orientation = "vertical")
plt.xlabel("Symptomatic Severity")
plt.ylabel("Insurance Charge")
plt.title("Net Cost Landscape")
plt.show()
net_cost_fig.savefig("net_cost_landscape.png", bbox_inches='tight')

#### Cost Sharing

In [ ]:
# Hyperparameters
small_enough = 0.1 # decides break point for value iteration algorithm
max_iter = 100 # decides break point for value iteration algorithm
gamma = 0.75 # discount rate for each time step
noise = 0.05 # probability of picking a different action than intended

# Define states - states[0] will be time step, states[1] will be either outside of care or inside of care
all_states = []
time_steps = 500
for i in np.arange(time_steps):
    for j in np.arange(2):
        all_states.append((i,j))
        
disease_processes = ["mild", "moderate", "severe"]
n_disease_processes = len(disease_processes)
cost_mus = np.arange(0.1, 1, 0.1)
n_cost_mus = len(cost_mus)
n_repeats = 10


cost_sharing_results = np.zeros((n_repeats, n_cost_mus, n_disease_processes))

for i in np.arange(n_disease_processes):
    disease_process = disease_processes[i]
    for j in np.arange(n_cost_mus):
        cost_mu = cost_mus[j]
        for k in np.arange(n_repeats):           
            # define disease process over time course of simulation - look at Designing Disease Processes section for more details
            initial_uncertainty = 0.05
            uncertainty_tau = 250
            symptoms_seq, uncertainty = diseaseSimulation(disease_process, time_steps, initial_uncertainty, uncertainty_tau)

            # define insurance cost process over time course of simulation as normally distributed around a stable mu depending on underlying disease
            cost_sigma = 0.05
            charges_seq = np.random.normal(cost_mu, cost_sigma, time_steps)
            charges_seq[charges_seq < 0] = 0
            charges_seq[charges_seq > 1] = 1

            # Define rewards
            rewards = {}
            for s in all_states:
                time_step = s[0]
                # if outside of care, reward is defined by only the symptomatic severity
                if s[1] == 0:
                    insurance_charge = 0
                    symptomatic_severity = symptoms_seq[time_step]
                # if inside of care, reward is defined by only the insurance charge
                else:
                    insurance_charge = charges_seq[time_step]
                    symptomatic_severity = 0
                [health_cost, insurance_cost, net_cost] = netCostCalculation(symptomatic_severity, insurance_charge)
                rewards[s] = -1*net_cost

            # Actions
            actions = {"outside": ('No Care', 'Seek Care'), "inside": ('Seek Care', 'Seek Care')}

            # Initial Policy
            policy = {}
            for s in all_states:
                # If outside of care, then initial policy will be to stay in that state until the very last action
                if s[1] == 0:
                    if s[0] < time_steps-2:
                        policy[s] = 'No Care'
                    elif s[0] == time_steps-2:
                        policy[s] = 'Seek Care'
                # If inside of care, then only possible action is to continue seeking care!
                elif s[0] <= time_steps-2:
                    policy[s] = 'Seek Care'

            # Initial Value Function
            V = {}
            for s in policy:
                V[s] = rewards[s]

            # Value Iteration Algorithm
            iteration = 0
            while True:
                biggest_change = 0
                for s in all_states:
                    if s in policy and s[0] < time_steps-2:
                        old_V = V[s]
                        new_V = -1000
                        if s[1] == 0:
                            care_state = "outside"
                        else:
                            care_state = "inside"
                        action_set = actions[care_state]
                        for a in action_set:
                            # next state for chosen action
                            if a == 'No Care' or (a == 'Seek Care' and care_state == "inside"):
                                nxt_state = [s[0]+1, s[1]]
                            if a == 'Seek Care' and care_state == "outside":
                                nxt_state = [s[0],s[1]+1]


                            # randomly choose an action - will allow selection of same action in order to be compatible with states inside of care
                            random_act = np.random.choice([i for i in action_set])
                            if random_act == 'No Care' or (random_act == 'Seek Care' and care_state == "inside"):
                                rand_state = [s[0]+1, s[1]]
                            if random_act == 'Seek Care' and care_state == "outside":
                                rand_state = [s[0],s[1]+1]

                            # calculate the value of this action
                            nxt_state = tuple(nxt_state)
                            rand_state = tuple(rand_state)
                            value = rewards[s] + gamma*((1-noise)*V[nxt_state]+(noise*V[rand_state]))
                            # if this action is the best so far, then keep it
                            if value > new_V:
                                new_V = value
                                policy[s] = a
                        # save the value of best action for the state
                        V[s] = new_V
                        # see if loop should break given the size of change
                        biggest_change = max(biggest_change, np.abs(old_V-V[s]))
                if biggest_change < small_enough or iteration > max_iter:
                    break
                iteration = iteration + 1

            seek_times = [time_seek for time_seek, v in policy.items() if v == 'Seek Care' and time_seek[1] == 0]
            seek_delay = seek_times[0][0]
            cost_sharing_results[k, j, i] = seek_delay

In [ ]:
all_means = np.mean(cost_sharing_results, axis = 0)
mild_means = all_means[:, 0]
moderate_means = all_means[:, 1]
severe_means = all_means[:, 2]

all_stds = np.std(cost_sharing_results, axis = 0)
mild_std_errs = all_stds[:, 0]/np.sqrt(n_repeats)
mild_lower = mild_means - mild_std_errs
mild_lower[mild_lower < 0] = 0
mild_upper = mild_means + mild_std_errs
mild_upper[mild_upper > 500] = 500
moderate_std_errs = all_stds[:, 1]/np.sqrt(n_repeats)
moderate_lower = moderate_means - moderate_std_errs
moderate_lower[moderate_lower < 0] = 0
moderate_upper = moderate_means + moderate_std_errs
moderate_upper[moderate_upper > 500] = 500
severe_std_errs = all_stds[:, 2]/np.sqrt(n_repeats)
severe_lower = severe_means - severe_std_errs
severe_lower[severe_lower < 0] = 0
severe_upper = severe_means + severe_std_errs
severe_upper[severe_upper > 500] = 500

overall_means = np.mean(all_means, axis = 1)
overall_stds = np.std(cost_sharing_results, axis = (0, 2))
overall_std_errs = overall_stds / np.sqrt(n_repeats*n_disease_processes)
overall_lower = overall_means - overall_std_errs
overall_lower[overall_lower < 0] = 0
overall_upper = overall_means + overall_std_errs
overall_upper[overall_upper > 500] = 500

In [ ]:
cost_sharing_fig = plt.figure(figsize = (10,10))
cost_sharing_fig.patch.set_facecolor("white")
ax = plt.axes()
plt.rcParams.update({'font.size': 16})
plt.xlabel("Average Insurance Charge")
plt.ylabel("Delay Before Seeking Care (time steps)")
plt.title("Effects of Cost Sharing")
plt.plot(cost_mus, mild_means, color = "blue", label = "mild disease")
plt.fill_between(cost_mus, mild_lower, mild_upper, color = "blue", alpha = 0.1)
plt.plot(cost_mus, moderate_means, color = "green", label = "moderate disease")
plt.fill_between(cost_mus, moderate_lower, moderate_upper, color = "green", alpha = 0.1)
plt.plot(cost_mus, severe_means, color = "red", label = "severe disease")
plt.fill_between(cost_mus, severe_lower, severe_upper, color = "red", alpha = 0.1)
plt.plot(cost_mus, overall_means, color = "black", linestyle = "--", label = "combined results")
plt.fill_between(cost_mus, overall_lower, overall_upper, color = "black", alpha = 0.1)
plt.ylim([-5, 505])
plt.legend(loc = "upper left")
plt.show()
cost_sharing_fig.savefig("cost_sharing.png", bbox_inches='tight')

#### Cost Uncertainty

In [ ]:
# Hyperparameters
small_enough = 0.1 # decides break point for value iteration algorithm
max_iter = 100 # decides break point for value iteration algorithm
gamma = 0.75 # discount rate for each time step
noise = 0.05 # probability of picking a different action than intended

# Define states - states[0] will be time step, states[1] will be either outside of care or inside of care
all_states = []
time_steps = 500
for i in np.arange(time_steps):
    for j in np.arange(2):
        all_states.append((i,j))
        
disease_processes = ["mild", "moderate", "severe"]
n_disease_processes = len(disease_processes)
cost_sigmas = np.arange(0.01, 0.21, 0.025)
n_cost_sigmas = len(cost_sigmas)
n_repeats = 10


cost_uncertainty_results = np.zeros((n_repeats, n_cost_sigmas, n_disease_processes))

for i in np.arange(n_disease_processes):
    disease_process = disease_processes[i]
    for j in np.arange(n_cost_sigmas):
        cost_sigma = cost_sigmas[j]
        for k in np.arange(n_repeats):           
            # define disease process over time course of simulation - look at Designing Disease Processes section for more details
            initial_uncertainty = 0.05
            uncertainty_tau = 250
            symptoms_seq, uncertainty = diseaseSimulation(disease_process, time_steps, initial_uncertainty, uncertainty_tau)

            # define insurance cost process over time course of simulation as normally distributed around a stable mu depending on underlying disease
            # cost_mus selected based on the Cost Sharing results to have most intermediate delay values
            if disease_process == "mild":
                cost_mu = 0.5
            elif disease_process == "moderate":
                cost_mu = 0.7
            else:
                cost_mu = 0.9
            
            charges_seq = np.random.normal(cost_mu, cost_sigma, time_steps)
            charges_seq[charges_seq < 0] = 0
            charges_seq[charges_seq > 1] = 1

            # Define rewards
            rewards = {}
            for s in all_states:
                time_step = s[0]
                # if outside of care, reward is defined by only the symptomatic severity
                if s[1] == 0:
                    insurance_charge = 0
                    symptomatic_severity = symptoms_seq[time_step]
                # if inside of care, reward is defined by only the insurance charge
                else:
                    insurance_charge = charges_seq[time_step]
                    symptomatic_severity = 0
                [health_cost, insurance_cost, net_cost] = netCostCalculation(symptomatic_severity, insurance_charge)
                rewards[s] = -1*net_cost

            # Actions
            actions = {"outside": ('No Care', 'Seek Care'), "inside": ('Seek Care', 'Seek Care')}

            # Initial Policy
            policy = {}
            for s in all_states:
                # If outside of care, then initial policy will be to stay in that state until the very last action
                if s[1] == 0:
                    if s[0] < time_steps-2:
                        policy[s] = 'No Care'
                    elif s[0] == time_steps-2:
                        policy[s] = 'Seek Care'
                # If inside of care, then only possible action is to continue seeking care!
                elif s[0] <= time_steps-2:
                    policy[s] = 'Seek Care'

            # Initial Value Function
            V = {}
            for s in policy:
                V[s] = rewards[s]

            # Value Iteration Algorithm
            iteration = 0
            while True:
                biggest_change = 0
                for s in all_states:
                    if s in policy and s[0] < time_steps-2:
                        old_V = V[s]
                        new_V = -1000
                        if s[1] == 0:
                            care_state = "outside"
                        else:
                            care_state = "inside"
                        action_set = actions[care_state]
                        for a in action_set:
                            # next state for chosen action
                            if a == 'No Care' or (a == 'Seek Care' and care_state == "inside"):
                                nxt_state = [s[0]+1, s[1]]
                            if a == 'Seek Care' and care_state == "outside":
                                nxt_state = [s[0],s[1]+1]


                            # randomly choose an action - will allow selection of same action in order to be compatible with states inside of care
                            random_act = np.random.choice([i for i in action_set])
                            if random_act == 'No Care' or (random_act == 'Seek Care' and care_state == "inside"):
                                rand_state = [s[0]+1, s[1]]
                            if random_act == 'Seek Care' and care_state == "outside":
                                rand_state = [s[0],s[1]+1]

                            # calculate the value of this action
                            nxt_state = tuple(nxt_state)
                            rand_state = tuple(rand_state)
                            value = rewards[s] + gamma*((1-noise)*V[nxt_state]+(noise*V[rand_state]))
                            # if this action is the best so far, then keep it
                            if value > new_V:
                                new_V = value
                                policy[s] = a
                        # save the value of best action for the state
                        V[s] = new_V
                        # see if loop should break given the size of change
                        biggest_change = max(biggest_change, np.abs(old_V-V[s]))
                if biggest_change < small_enough or iteration > max_iter:
                    break
                iteration = iteration + 1

            seek_times = [time_seek for time_seek, v in policy.items() if v == 'Seek Care' and time_seek[1] == 0]
            seek_delay = seek_times[0][0]
            cost_uncertainty_results[k, j, i] = seek_delay

In [ ]:
all_means = np.mean(cost_uncertainty_results, axis = 0)
mild_means = all_means[:, 0]
moderate_means = all_means[:, 1]
severe_means = all_means[:, 2]

all_stds = np.std(cost_uncertainty_results, axis = 0)
mild_std_errs = all_stds[:, 0]/np.sqrt(n_repeats)
mild_lower = mild_means - mild_std_errs
mild_lower[mild_lower < 0] = 0
mild_upper = mild_means + mild_std_errs
mild_upper[mild_upper > 500] = 500
moderate_std_errs = all_stds[:, 1]/np.sqrt(n_repeats)
moderate_lower = moderate_means - moderate_std_errs
moderate_lower[moderate_lower < 0] = 0
moderate_upper = moderate_means + moderate_std_errs
moderate_upper[moderate_upper > 500] = 500
severe_std_errs = all_stds[:, 2]/np.sqrt(n_repeats)
severe_lower = severe_means - severe_std_errs
severe_lower[severe_lower < 0] = 0
severe_upper = severe_means + severe_std_errs
severe_upper[severe_upper > 500] = 500

mild_percs = 100*(mild_means - mild_means[0])/mild_means[0]
mild_lower_percs = 100*(mild_lower - mild_means[0])/mild_means[0]
mild_upper_percs = 100*(mild_upper - mild_means[0])/mild_means[0]

moderate_percs = 100*(moderate_means - moderate_means[0])/moderate_means[0]
moderate_lower_percs = 100*(moderate_lower - moderate_means[0])/moderate_means[0]
moderate_upper_percs = 100*(moderate_upper - moderate_means[0])/moderate_means[0]

severe_percs = 100*(severe_means - severe_means[0])/severe_means[0]
severe_lower_percs = 100*(severe_lower - severe_means[0])/severe_means[0]
severe_upper_percs = 100*(severe_upper - severe_means[0])/severe_means[0]

In [ ]:
cost_uncertainty_fig = plt.figure(figsize = (10,10))
cost_uncertainty_fig.patch.set_facecolor("white")
ax = plt.axes()
plt.rcParams.update({'font.size': 16})
ax1 = plt.subplot(2,1,1)
plt.ylabel("Delay Before Seeking Care \n (time steps)")
plt.title("Effects of Cost Uncertainty")
plt.plot(cost_sigmas, mild_means, color = "blue", label = "mild disease")
plt.fill_between(cost_sigmas, mild_lower, mild_upper, color = "blue", alpha = 0.1)
plt.plot(cost_sigmas, moderate_means, color = "green", label = "moderate disease")
plt.fill_between(cost_sigmas, moderate_lower, moderate_upper, color = "green", alpha = 0.1)
plt.plot(cost_sigmas, severe_means, color = "red", label = "severe disease")
plt.fill_between(cost_sigmas, severe_lower, severe_upper, color = "red", alpha = 0.1)
plt.ylim([-5, 505])
plt.legend(loc = "upper left")
ax2 = plt.subplot(2,1,2)
plt.hlines(0, min(cost_sigmas), max(cost_sigmas), linestyle = "--", color = "black")
plt.plot(cost_sigmas, mild_percs, color = "blue", label = "mild disease")
plt.fill_between(cost_sigmas, mild_lower_percs, mild_upper_percs, color = "blue", alpha = 0.1)
plt.plot(cost_sigmas, moderate_percs, color = "green", label = "moderate disease")
plt.fill_between(cost_sigmas, moderate_lower_percs, moderate_upper_percs, color = "green", alpha = 0.1)
plt.plot(cost_sigmas, severe_percs, color = "red", label = "severe disease")
plt.fill_between(cost_sigmas, severe_lower_percs, severe_upper_percs, color = "red", alpha = 0.1)
plt.xlabel("Insurance Charge Uncertainty")
plt.ylabel("Percent Change in Delay \n from Minimal Cost Uncertainty")
plt.show()
cost_uncertainty_fig.savefig("cost_uncertainty.png", bbox_inches='tight')

#### Health Uncertainty

In [ ]:
# Hyperparameters
small_enough = 0.1 # decides break point for value iteration algorithm
max_iter = 100 # decides break point for value iteration algorithm
gamma = 0.75 # discount rate for each time step
noise = 0.05 # probability of picking a different action than intended

# Define states - states[0] will be time step, states[1] will be either outside of care or inside of care
all_states = []
time_steps = 500
for i in np.arange(time_steps):
    for j in np.arange(2):
        all_states.append((i,j))
        
disease_processes = ["mild", "moderate", "severe"]
n_disease_processes = len(disease_processes)
health_sigmas = np.arange(0.01, 0.285, 0.025)
n_health_sigmas = len(health_sigmas)
n_repeats = 10


health_uncertainty_results = np.zeros((n_repeats, n_health_sigmas, n_disease_processes))

for i in np.arange(n_disease_processes):
    disease_process = disease_processes[i]
    for j in np.arange(n_health_sigmas):
        health_sigma = health_sigmas[j]
        for k in np.arange(n_repeats):           
            # define disease process over time course of simulation - look at Designing Disease Processes section for more details
            initial_uncertainty = health_sigma
            uncertainty_tau = 250
            symptoms_seq, uncertainty = diseaseSimulation(disease_process, time_steps, initial_uncertainty, uncertainty_tau)

            # define insurance cost process over time course of simulation as normally distributed around a stable mu depending on underlying disease
            # cost_mus selected based on the Cost Sharing results to have most intermediate delay values
            if disease_process == "mild":
                cost_mu = 0.5
            elif disease_process == "moderate":
                cost_mu = 0.7
            else:
                cost_mu = 0.9
            # cost_sigma selected based on the Cost Uncertainty results to have most intermediate delay values
            cost_sigma = 0.06
            charges_seq = np.random.normal(cost_mu, cost_sigma, time_steps)
            charges_seq[charges_seq < 0] = 0
            charges_seq[charges_seq > 1] = 1

            # Define rewards
            rewards = {}
            for s in all_states:
                time_step = s[0]
                # if outside of care, reward is defined by only the symptomatic severity
                if s[1] == 0:
                    insurance_charge = 0
                    symptomatic_severity = symptoms_seq[time_step]
                # if inside of care, reward is defined by only the insurance charge
                else:
                    insurance_charge = charges_seq[time_step]
                    symptomatic_severity = 0
                [health_cost, insurance_cost, net_cost] = netCostCalculation(symptomatic_severity, insurance_charge)
                rewards[s] = -1*net_cost

            # Actions
            actions = {"outside": ('No Care', 'Seek Care'), "inside": ('Seek Care', 'Seek Care')}

            # Initial Policy
            policy = {}
            for s in all_states:
                # If outside of care, then initial policy will be to stay in that state until the very last action
                if s[1] == 0:
                    if s[0] < time_steps-2:
                        policy[s] = 'No Care'
                    elif s[0] == time_steps-2:
                        policy[s] = 'Seek Care'
                # If inside of care, then only possible action is to continue seeking care!
                elif s[0] <= time_steps-2:
                    policy[s] = 'Seek Care'

            # Initial Value Function
            V = {}
            for s in policy:
                V[s] = rewards[s]

            # Value Iteration Algorithm
            iteration = 0
            while True:
                biggest_change = 0
                for s in all_states:
                    if s in policy and s[0] < time_steps-2:
                        old_V = V[s]
                        new_V = -1000
                        if s[1] == 0:
                            care_state = "outside"
                        else:
                            care_state = "inside"
                        action_set = actions[care_state]
                        for a in action_set:
                            # next state for chosen action
                            if a == 'No Care' or (a == 'Seek Care' and care_state == "inside"):
                                nxt_state = [s[0]+1, s[1]]
                            if a == 'Seek Care' and care_state == "outside":
                                nxt_state = [s[0],s[1]+1]


                            # randomly choose an action - will allow selection of same action in order to be compatible with states inside of care
                            random_act = np.random.choice([i for i in action_set])
                            if random_act == 'No Care' or (random_act == 'Seek Care' and care_state == "inside"):
                                rand_state = [s[0]+1, s[1]]
                            if random_act == 'Seek Care' and care_state == "outside":
                                rand_state = [s[0],s[1]+1]

                            # calculate the value of this action
                            nxt_state = tuple(nxt_state)
                            rand_state = tuple(rand_state)
                            value = rewards[s] + gamma*((1-noise)*V[nxt_state]+(noise*V[rand_state]))
                            # if this action is the best so far, then keep it
                            if value > new_V:
                                new_V = value
                                policy[s] = a
                        # save the value of best action for the state
                        V[s] = new_V
                        # see if loop should break given the size of change
                        biggest_change = max(biggest_change, np.abs(old_V-V[s]))
                if biggest_change < small_enough or iteration > max_iter:
                    break
                iteration = iteration + 1

            seek_times = [time_seek for time_seek, v in policy.items() if v == 'Seek Care' and time_seek[1] == 0]
            seek_delay = seek_times[0][0]
            health_uncertainty_results[k, j, i] = seek_delay

In [ ]:
all_means = np.mean(health_uncertainty_results, axis = 0)
mild_means = all_means[:, 0]
moderate_means = all_means[:, 1]
severe_means = all_means[:, 2]

all_stds = np.std(health_uncertainty_results, axis = 0)
mild_std_errs = all_stds[:, 0]/np.sqrt(n_repeats)
mild_lower = mild_means - mild_std_errs
mild_lower[mild_lower < 0] = 0
mild_upper = mild_means + mild_std_errs
mild_upper[mild_upper > 500] = 500
moderate_std_errs = all_stds[:, 1]/np.sqrt(n_repeats)
moderate_lower = moderate_means - moderate_std_errs
moderate_lower[moderate_lower < 0] = 0
moderate_upper = moderate_means + moderate_std_errs
moderate_upper[moderate_upper > 500] = 500
severe_std_errs = all_stds[:, 2]/np.sqrt(n_repeats)
severe_lower = severe_means - severe_std_errs
severe_lower[severe_lower < 0] = 0
severe_upper = severe_means + severe_std_errs
severe_upper[severe_upper > 500] = 500

mild_percs = 100*(mild_means - mild_means[0])/mild_means[0]
mild_lower_percs = 100*(mild_lower - mild_means[0])/mild_means[0]
mild_upper_percs = 100*(mild_upper - mild_means[0])/mild_means[0]

moderate_percs = 100*(moderate_means - moderate_means[0])/moderate_means[0]
moderate_lower_percs = 100*(moderate_lower - moderate_means[0])/moderate_means[0]
moderate_upper_percs = 100*(moderate_upper - moderate_means[0])/moderate_means[0]

severe_percs = 100*(severe_means - severe_means[0])/severe_means[0]
severe_lower_percs = 100*(severe_lower - severe_means[0])/severe_means[0]
severe_upper_percs = 100*(severe_upper - severe_means[0])/severe_means[0]

In [ ]:
health_uncertainty_fig = plt.figure(figsize = (10,10))
health_uncertainty_fig.patch.set_facecolor("white")
ax1 = plt.subplot(2,1,1)
plt.rcParams.update({'font.size': 16})
plt.ylabel("Delay Before Seeking Care \n (time steps)")
plt.plot(health_sigmas, mild_means, color = "blue", label = "mild disease")
plt.fill_between(health_sigmas, mild_lower, mild_upper, color = "blue", alpha = 0.1)
plt.plot(health_sigmas, moderate_means, color = "green", label = "moderate disease")
plt.fill_between(health_sigmas, moderate_lower, moderate_upper, color = "green", alpha = 0.1)
plt.plot(health_sigmas, severe_means, color = "red", label = "severe disease")
plt.fill_between(health_sigmas, severe_lower, severe_upper, color = "red", alpha = 0.1)
plt.ylim([-5, 505])
plt.legend(loc = "upper left")
ax2 = plt.subplot(2,1,2)
plt.hlines(0, min(health_sigmas), max(health_sigmas), linestyle = "--", color = "black")
plt.plot(health_sigmas, mild_percs, color = "blue", label = "mild disease")
plt.fill_between(health_sigmas, mild_lower_percs, mild_upper_percs, color = "blue", alpha = 0.1)
plt.plot(health_sigmas, moderate_percs, color = "green", label = "moderate disease")
plt.fill_between(health_sigmas, moderate_lower_percs, moderate_upper_percs, color = "green", alpha = 0.1)
plt.plot(health_sigmas, severe_percs, color = "red", label = "severe disease")
plt.fill_between(health_sigmas, severe_lower_percs, severe_upper_percs, color = "red", alpha = 0.1)
plt.xlabel("Initial Health Status Uncertainty")
plt.ylabel("Percent Change in Delay \n from Minimal Health Uncertainty")
plt.show()
health_uncertainty_fig.savefig("health_uncertainty.png", bbox_inches='tight')

In [ ]:
cost_sigmas = np.arange(0.01, 0.21, 0.025)
cost_sigmas

In [ ]:
health_sigmas = np.arange(0.01, 0.285, 0.025)
health_sigmas